In [9]:
import pandas as pd
import numpy as np
import folium
import skmob
import os

from gnact import utils, clust, plotting, network

In [10]:
df = pd.read_csv('/Users/patrickmaus/Documents/projects/cabs/cabspottingdata/new_enarwee.txt',
                 names=['lat', 'lon', 'fare', 'time'],
                 delimiter=' ')
df['time'] = pd.to_datetime(df['time'], unit='s').copy()
name = 'enarwee'
df['cab_name'] = name
df = df.sort_values('time').copy()
df = df.reset_index(drop=True)
df = df.reset_index().rename(columns={'index':'id'})
df.head()

,id,lat,lon,fare,time,cab_name
0,0,37.75155,-122.39446,0,2008-05-17 12:30:46,enarwee
1,1,37.75171,-122.39469,0,2008-05-17 12:31:46,enarwee
2,2,37.75142,-122.39487,0,2008-05-17 12:32:46,enarwee
3,3,37.75138,-122.39504,0,2008-05-17 12:33:41,enarwee
4,4,37.75140,-122.39498,0,2008-05-17 12:34:41,enarwee


In [11]:
df['prev_fare'] = df['fare'].shift(1)
df['next_fare'] = df['fare'].shift(-1)

In [12]:
df_pickups = df[(df['fare'] != df['prev_fare']) & (df['fare'] == 1)]
df_pickups

,id,lat,lon,fare,time,cab_name,prev_fare,next_fare
27,27,37.77771,-122.43162,1,2008-05-17 12:56:14,enarwee,0.0,0.0
48,48,37.79240,-122.43452,1,2008-05-17 13:16:02,enarwee,0.0,1.0
80,80,37.77875,-122.47713,1,2008-05-17 13:46:21,enarwee,0.0,1.0
136,136,37.79056,-122.41049,1,2008-05-17 14:42:40,enarwee,0.0,1.0
154,154,37.79294,-122.41007,1,2008-05-17 14:58:15,enarwee,0.0,1.0
...,...,...,...,...,...,...,...,...
24129,24129,37.78730,-122.42310,1,2008-06-09 17:17:01,enarwee,0.0,1.0
24152,24152,37.77997,-122.48542,1,2008-06-09 17:39:37,enarwee,0.0,1.0
24181,24181,37.78677,-122.42010,1,2008-06-09 18:06:21,enarwee,0.0,1.0
24193,24193,37.78416,-122.44125,1,2008-06-09 18:15:12,enarwee,0.0,1.0


In [13]:
df_dropoffs = df[(df['fare'] != df['next_fare']) & (df['fare'] == 1)]
df_dropoffs

,id,lat,lon,fare,time,cab_name,prev_fare,next_fare
27,27,37.77771,-122.43162,1,2008-05-17 12:56:14,enarwee,0.0,0.0
53,53,37.78151,-122.42048,1,2008-05-17 13:20:41,enarwee,1.0,0.0
95,95,37.78823,-122.40261,1,2008-05-17 14:01:45,enarwee,1.0,0.0
141,141,37.78344,-122.40280,1,2008-05-17 14:47:27,enarwee,1.0,0.0
157,157,37.79378,-122.39635,1,2008-05-17 15:01:17,enarwee,1.0,0.0
...,...,...,...,...,...,...,...,...
24144,24144,37.78266,-122.50384,1,2008-06-09 17:32:14,enarwee,1.0,0.0
24171,24171,37.78488,-122.41316,1,2008-06-09 17:57:09,enarwee,1.0,0.0
24187,24187,37.78324,-122.43271,1,2008-06-09 18:11:51,enarwee,1.0,0.0
24203,24203,37.79474,-122.40380,1,2008-06-09 18:25:40,enarwee,1.0,0.0


In [14]:
df_clusts = clust.calc_clusts(df_dropoffs, method='optics', eps_km=.5, min_samp=5)


/Users/patrickmaus/PycharmProjects/gnact/clust.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['time'].dt.floor('min')


In [15]:
df_centers = clust.calc_centers(df_clusts)
df_centers.sort_values('total_clust_count')

,clust_id,time_min,time_max,total_clust_count,average_lat,average_lon,average_dist_from_center,time_diff
63,63,2008-05-24 02:26:00,2008-06-02 23:27:00,5,37.615354,-122.390694,0.092063,9 days 21:01:00
24,24,2008-05-17 18:38:00,2008-06-06 20:56:00,5,37.790720,-122.419162,0.088500,20 days 02:18:00
45,45,2008-05-22 17:28:00,2008-06-06 19:42:00,5,37.785940,-122.429348,0.086082,15 days 02:14:00
47,47,2008-05-22 01:00:00,2008-06-04 15:46:00,5,37.749682,-122.412812,0.210209,13 days 14:46:00
9,9,2008-05-21 06:15:00,2008-06-04 03:41:00,5,37.751958,-122.418826,0.089841,13 days 21:26:00
...,...,...,...,...,...,...,...,...
29,29,2008-05-17 15:01:00,2008-06-05 17:36:00,11,37.793736,-122.396702,0.091720,19 days 02:35:00
31,31,2008-05-25 02:23:00,2008-06-07 00:44:00,12,37.795754,-122.416795,0.179973,12 days 22:21:00
44,44,2008-05-22 01:44:00,2008-06-08 14:53:00,13,37.772655,-122.412730,0.229999,17 days 13:09:00
53,53,2008-05-24 22:49:00,2008-06-09 14:29:00,14,37.769317,-122.448089,0.159301,15 days 15:40:00


In [16]:
plotting.plot_clusters(df_clusts, df_centers)

Plotted 64 total clusters.
